## Make Allan Deviation Plot

In [ ]:
import obspy as obs
import matplotlib.pyplot as plt
import numpy as np
import allantools

from andbro__querrySeismoData import __querrySeismoData
from andbro__savefig import __savefig

## Configurations

In [ ]:
## Configurations

config = {}

config['tbeg'] = obs.UTCDateTime("2023-01-12 00:00")
config['tend'] = config['tbeg']+ 86400

# config['seed'] = "BW.DROMY..FJZ"
config['seed'] = "BW.ROMY.10.BJZ"

config['repository'] = "george"

# config['outpath'] = f''
# config['outfile'] = f''
config['outfigs'] = f'./'

config['net'], config['sta'], config['loc'], config['cha'] = config['seed'].split(".")

## ALLAN
config['errorbars'] = False
config['relative'] = False

## Load Data

In [ ]:
st, inv = __querrySeismoData(    
                            seed_id=config['seed'],
                            starttime=config['tbeg']-10,
                            endtime=config['tend']+10,
                            repository=config['repository'],
                            path=None,
                            restitute=False,
                            detail=None,
                            fill_value=None,
)

In [ ]:
st.detrend("demean")

st.remove_sensitivity(inv)

# st.trim(config['tbeg'], config['tend'])

In [ ]:
st.plot();

## Compute Allan Dev

In [ ]:
def __compute_allan_deviations(config, st):
    
    ## set interval
    inter = 3600*2
    
    ## sampling rate
    df = st[0].stats.sampling_rate
    
    ## looping hourly
    t1, t2 = config['tbeg'], config['tbeg']+inter
    
    allans,traces = [], []
    nn = 0
    while t2 <= config['tend']:
        
        print(str(nn).rjust(2,"0"),": ", t1, t2)
        tmp = st.copy().trim(t1, t2)
        traces.append(tmp[0].data)
        
        ## normal allan deviation
        ## allan = (taus1, adevs1, errors, ns) 
        allan = allantools.adev(tmp[0].data, 
                                data_type='freq', 
                                rate=df, 
                                taus='all', ## all, octave, decade
                                )
        allans.append(allan)
        t1 += inter
        t2 += inter
        nn += 1
        
    return allans, traces

allan, traces = __compute_allan_deviations(config, st)

## Plotting

In [ ]:
def __makeplot_allan_deviation_overview(config, st, allan, traces):

    from numpy import shape, linspace
    
    taus, adev, errors, ns = allan[0], allan[1], allan[2], allan[3]

    day = st[0].stats.starttime.date
    tt1 = str(st[0].stats.starttime.time)[:8]
    tt2 = str(st[0].stats.endtime.time)[:8]  
    
    if config['relative']:
        adev = adev / 7.3e-5
        
    ## -------------------------------------------
    fig, ax = plt.subplots(1,2, figsize=(18,6))

    plt.subplots_adjust(wspace=0.15)
    
    font = 14

    cols = plt.cm.viridis(linspace(0,1,shape(traces)[0]+1))
    cols = plt.cm.rainbow(linspace(0,1,shape(traces)[0]+1))
    
    ## ___________________________________
    ## PLOT 1
    
    for n, al in enumerate(allan):
        taus, adev, errors, ns = al[0], al[1], al[2], al[3]

        ax[0].plot(taus, adev, color=cols[n], label=f"{config['sta']}.{config['cha']}")
        
        
    if config['relative']:
        ax[0].set_ylabel(r"Allan Deviation  $\Omega$/$\Omega_E$", fontsize=font)
    else:
        ax[0].set_ylabel(r"Allan Deviation $\sigma_{\tau}$ [rad/s]", fontsize=font)

    ax[0].set_xlabel(r"$\tau$ [s]", fontsize=font)
    
    ax[0].set_xscale('log')
    ax[0].set_yscale('log')

    ax[0].grid(True, which="both", ls="--")

    ax[0].tick_params(labelsize=font-1)
    
#     ax[0].set_title(f"Allan Deviation for {config['net']}.{config['cha']} on {day}  [{tt1}-{tt2}]", fontsize=font+3)

    
    ## ___________________________________
    ## PLOT 2
    
    norm_st_max = max(st[0].data)
    timeaxis = linspace(0, 120, len(traces[0]))
    
    for m, tr in enumerate(traces):
        
        norm_tr_max = max(tr)
        
        ax[1].plot(timeaxis, tr/norm_st_max + m, color=cols[m])
    
    ax[1].set_yticks(linspace(0,11,shape(traces)[0]))
    ax[1].set_yticklabels([str(int(tt)).rjust(2,"0")+":00" for tt in linspace(0,22,shape(traces)[0])])
    
    ax[1].set_ylim(-1, shape(traces)[0])
    
    ax[1].set_xlabel("Time [min]", fontsize=font)
    
    ax[1].tick_params(labelsize=font-1)

    plt.show();
    return fig 

fig = __makeplot_allan_deviation_overview(config, st, allan, traces)

In [ ]:
__savefig(fig, outpath=config['outfigs'], outname=f"AllanDev_{config['sta']}_{config['cha']}_{str(config['tbeg'].date).replace('-','')}", mode="png", dpi=300)